# PetFinder Image and Tabular Regression

## [목차]
1. 모듈 임포트 및 하이퍼 파라미터 설정
2. 데이터 읽기
3. 데이터 전처리 클래스 및 함수 작성
4. 훈련 준비 - 기학습 모델 불러오기
5. 훈련 수행
6. 훈련 모델을 바탕으로 테스트
7. 훈련된 모델을 저장
8. 파인튜닝 훈련한 모델을 로드해서 추론 하기


<br>
### [참고 자료]



## [사전작업] 기반 라이브러리 설치

In [ ]:
# !pip install torch==1.7.0
# !pip install tqdm
# !pip install numpy==1.19.5
# !pip install seqeval
# !pip install scikit-learn==0.23.2
# !pip install pandas==1.2.4
# !pip install opencv-python-headless==4.5.2.54
# !pip install transformers==4.6.1
# !pip install albumentations==1.0.1
# !pip install timm
# # !pip install pydicom==2.1.2
# !pip install matplotlib
# !pip install tez

In [ ]:
# !nvidia-smi
# !lsb_release -a
# !python --version

In [ ]:
# import pynvml

# pynvml.nvmlInit()
# handle = pynvml.nvmlDeviceGetHandleByIndex(0)
# device_name = pynvml.nvmlDeviceGetName(handle)

# if device_name != b'Tesla T4':
#   raise Exception("""
#     Unfortunately this instance does not have a T4 GPU.
    
#     Please make sure you've configured Colab to request a GPU instance type.
    
#     Sometimes Colab allocates a Tesla K80 instead of a T4. Resetting the instance.

#     If you get a K80 GPU, try Runtime -> Reset all runtimes...
#   """)
# else:
#   print('Woo! You got the right kind of GPU!')

# [1] swin_large_patch4_window12_384 + SVR

https://www.kaggle.com/abhishek/tez-pawpular-training

https://www.kaggle.com/abhishek/tez-pawpular-swin-ference

https://www.kaggle.com/cdeotte/rapids-svr-boost-17-8

https://www.kaggle.com/manabendrarout/transformers-classifier-method-starter-infer

https://www.kaggle.com/manabendrarout/transformers-classifier-method-starter-train

https://www.kaggle.com/markerkor/petfindder-updated-rules-cheol-5fold/notebook






## 1. 모듈 임포트 및 하이퍼 파라미터 설정

In [ ]:
import sys
sys.path.append("../input/tez-lib/")
sys.path.append("../input/timmmaster/")

import tez
from tez.callbacks import EarlyStopping
import sys
import platform
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import cv2
from PIL import Image
import random
import glob
import gc
import copy
from math import ceil
import math
from albumentations import Compose, Normalize, HueSaturationValue, RandomBrightnessContrast, HorizontalFlip, VerticalFlip, Rotate, ShiftScaleRotate, RGBShift, augmentations
from albumentations import Resize as Resize_alb
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split, KFold
import torch
import shutil
from torchvision.transforms import ToTensor
import time
import timm
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.distributed import DistributedSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau, LambdaLR, StepLR, CosineAnnealingWarmRestarts

import warnings
warnings.simplefilter('ignore')

import cuml, pickle
from cuml.svm import SVR
print('RAPIDS version',cuml.__version__,'\n')

In [ ]:
# MODEL_NAME = 'tf_efficientnet_b0_ns'
# MODEL_NAME = 'vit_large_patch32_384'
MODEL_NAME = 'swin_large_patch4_window12_384'
IMAGE_SIZE = 384 # swin large should be 384
BATCH_SIZE = 4# 32 
EPOCHS = 10 # 20
SEED = 1234
LEARNING_RATE = 1e-4 #5e-5  
WORKERS = 8 
# drop_last = True
NUM_FOLDS = 5
EARLY_STOP = 5
DO_CLIP_GRAD = False
USE_SWIN_SVR = False

# AUGMENTS = Compose(
#     [
#         Resize_alb(IMAGE_SIZE, IMAGE_SIZE, p=1),
#         HueSaturationValue(
#             hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5
#         ),
#         RandomBrightnessContrast(
#             brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5
#         ),
#         Normalize(
#             mean=[0.500, 0.500, 0.500],
#             std=[0.270, 0.270, 0.270],
#             max_pixel_value=255.0,
#             p=1.0,
#         ),
#     ],
#     p=1.0,
# )

AUGMENTS = Compose(
    [
        Resize_alb(IMAGE_SIZE, IMAGE_SIZE, p=1),
        Normalize(
            mean=[0.500, 0.500, 0.500],
            std=[0.270, 0.270, 0.270],
            max_pixel_value=255.0,
            p=1.0,
        ),
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        Rotate(limit=180, p=0.7),
        ShiftScaleRotate(
                shift_limit = 0.1, scale_limit=0.1, rotate_limit=45, p=0.5
            ),
        HueSaturationValue(
            hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5
        ),
        RandomBrightnessContrast(
            brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5
        ),

    ],
    p=1.0,
)

AUGMENTS2 = Compose(
    [
        Resize_alb(IMAGE_SIZE, IMAGE_SIZE, p=1),
        Normalize(
            mean=[0.500, 0.500, 0.500],
            std=[0.270, 0.270, 0.270],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

AUGMENTS3 = Compose(
    [
        Resize_alb(IMAGE_SIZE, IMAGE_SIZE, p=1),
        ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        RandomBrightnessContrast(p=0.5),
        
        
        Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

# Brightness(), Contrast(), Hue(), Saturation()

AUGMENTS4 = Compose(
    [
        Resize_alb(IMAGE_SIZE, IMAGE_SIZE, p=1),
        ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        RandomBrightnessContrast(p=0.5),
        augmentations.transforms.Cutout(num_holes=8, max_h_size=8, max_w_size=8, fill_value=0, always_apply=False, p=0.5),
        
        Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

DO_KFOLD_TRAINING = False
# DATA_IN_PATH = '/content/drive/MyDrive/Colab_Notebooks/data/petfinder/'
# SAVE_MODEL_PATH = '/content/drive/MyDrive/Portfolio_for_AI/'
DATA_IN_PATH = '../input/petfinder-pawpularity-score/'
SAVE_MODEL_PATH = '../input/petfinder-trained-model-v2/'
DO_SVR_TRAIN = False

DENSE_FEATURES = [
    'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
    'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
]

# Make results reproducible
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True
    
seed_everything(SEED)

# Disable Debug APIs for Final Training
torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

## 2. 데이터 읽기

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
dataset_df = pd.read_csv(f"{DATA_IN_PATH}train.csv")

print("텍스트 회귀 학습_검증 데이터 개수: {}".format(len(dataset_df)))

In [ ]:
pd.set_option('display.max_columns', None)
print(dataset_df)

## 3. 데이터 전처리 클래스 및 함수 작성

In [ ]:
class PawpularDataset(Dataset):
    def __init__(self, image_paths, dense_features, targets=None, augmentations=None):
        self.image_paths = image_paths
        self.dense_features = dense_features
        self.targets = targets
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):
        image = cv2.imread(self.image_paths[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
            
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        
        features = self.dense_features[item, :]

        if self.targets is not None:
            targets = self.targets[item]
            return (torch.tensor(image, dtype=torch.float), 
                    torch.tensor(features, dtype=torch.float),
                    torch.tensor(targets, dtype=torch.float))
        else:
            return (torch.tensor(image, dtype=torch.float), 
                    torch.tensor(features, dtype=torch.float))
        
class PawpularDataset2:
    def __init__(self, image_paths, dense_features, targets=None, augmentations=None):
        self.image_paths = image_paths
        self.dense_features = dense_features
        self.targets = targets
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):
        image = cv2.imread(self.image_paths[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
            
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        
        features = self.dense_features[item, :]
        targets = self.targets[item]
        
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "features": torch.tensor(features, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.float),
        }


### 3-1) 위에서 작성한 전처리 함수들 테스트해보기 

In [ ]:
tmp_df = dataset_df.iloc[:100]
img_paths = [f"{DATA_IN_PATH}train/{x}.jpg" for x in tmp_df["Id"].values]
# img_paths += [f"{DATA_IN_PATH}cropped_train/{x}.jpg" for x in tmp_df["Id"].values]
# tmp_df = pd.concat([tmp_df, tmp_df], axis=0)
tmp_dataset = PawpularDataset2(
    image_paths=img_paths,
    dense_features=tmp_df[DENSE_FEATURES].values,
    targets=tmp_df.Pawpularity.values/100.0,
    augmentations=AUGMENTS,
)
print(tmp_dataset[10])

## 4. 훈련 준비 - 기학습 모델 불러오기

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_fn = nn.MSELoss()
scaler = GradScaler()
softmax_fn = nn.Softmax(dim=1)

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

class RMSELoss(torch.nn.Module):
    def __init__(self):
        super(RMSELoss,self).__init__()
        self.criterion = nn.MSELoss()
 
    def forward(self,x,y):
        x, y = x.type(torch.FloatTensor), y.type(torch.FloatTensor)
        loss = torch.sqrt(self.criterion(x, y)).type(torch.FloatTensor)
        return loss

class PawpularModel(nn.Module):
    def __init__(self, model_name, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=3)
        # self.model.classifier = nn.Linear(self.model.classifier.in_features, 128) # effnet
        self.model.head = nn.Linear(self.model.head.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.dense1 = nn.Linear(140, 64)
        self.dense2 = nn.Linear(64, 1)
        self.activation = nn.Sigmoid()

    def forward(self, image, features):
        x = self.model(image)
        x = self.dropout(x)
        x = torch.cat([x, features], dim=1)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.activation(x)
        return x

class PawpularModel2(nn.Module):
    def __init__(self, model_name, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=3)
        self.model.head = nn.Linear(self.model.head.in_features, 128)
        self.fc = nn.Sequential(
            nn.Linear(140, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
        self.dropout = nn.Dropout(0.2)
        self.activation = nn.Sigmoid()
    
    def forward(self, image, dense):
        embeddings = self.model(image)
        x = self.dropout(embeddings)
        x = torch.cat([x, dense], dim=1)
        x = self.fc(x)
        x = self.activation(x)
        return x

 
class PawpularModel3(tez.Model):
    def __init__(self, model_name, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=3)
        self.model.head = nn.Linear(self.model.head.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.dense1 = nn.Linear(140, 64)
        self.dense2 = nn.Linear(64, 1)

    def forward(self, image, features, targets=None):
        x1 = self.model(image)
        x = self.dropout(x1)
        x = torch.cat([x, features], dim=1)
        x = self.dense1(x)
        x = self.dense2(x)
        
        x = torch.cat([x, x1, features], dim=1)
        return x, 0, {}
    


# # https://www.kaggle.com/h053473666/siim-covid19-efnb7-train-study
# class EfficientNetModel(nn.Module):
#     """
#     Model Class for EfficientNet Model
#     """
#     def __init__(self, num_classes=4, model_name=MODEL_NAME, pretrained=True):
#         super(EfficientNetModel, self).__init__()
#         self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=3)
#         # weights='imagenet'
#         #self.model.global_pool = nn.AdaptiveAvgPool2d(self.model.classifier.in_features)
#         self.model.classifier = nn.Linear(self.model.classifier.in_features, num_classes)
#         # self.soft = nn.Softmax(dim=1)
        
#     def forward(self, x):
#         x = self.model(x)
#         return x
    

# class NFNetModel(nn.Module):
#     """
#     Model Class for EfficientNet Model
#     """
#     def __init__(self, num_classes=4, model_name=MODEL_NAME, pretrained=True):
#         super(NFNetModel, self).__init__()
#         self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=3)
#         self.model.head.fc = nn.Linear(self.model.head.fc.in_features, num_classes)
#         # self.soft = nn.Softmax(dim=1)
        
#     def forward(self, x):
#         x = self.model(x)
#         return x


## 5. 훈련 수행 함수 

- 훈련셋에 대해서 정해진 epoch 수 만큼 훈련을 하면서 정확도와 F1스코어를 출력

- 훈련셋 minibatch 마다 validset evaluation 수행하여 최적 모델 저장

In [ ]:
def evaluate(model, test_dataset, phrase='test'):
  test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, 
                           num_workers = WORKERS, pin_memory = True)
 
  model.eval()
 
  total_loss = 0.0
  batches = 0
  total_cnt = 0
  correct_cnt = 0
  all_labels = list()
  all_preds = list()
  num_batches = len(test_loader)
  if phrase.lower() != 'test':
    phrase = 'Validation: '
  else:
    phrase = 'Testing: '
  with torch.no_grad():
    progress_bar = tqdm(test_loader, desc=phrase, unit='batch')
    for i, mini_batch in enumerate(progress_bar):
      inputs = {'image': mini_batch[0].to(device, dtype=torch.float), 
                'features': mini_batch[1].to(device, dtype=torch.float)}
      outputs = model(**inputs)
      
      loss = loss_fn(outputs, mini_batch[2].to(device, dtype=torch.float))
 
      batches += 1
      total_loss += loss.item()
      avg_loss = total_loss / batches
 
      if 'test' not in phrase.lower():
        progress_bar.set_postfix_str('|val_loss|=%.4f' % (avg_loss))
      else:
        progress_bar.set_postfix_str('|test_loss|=%.4f' % (avg_loss))
      torch.cuda.empty_cache()

    del inputs, outputs
    gc.collect()
    torch.cuda.empty_cache()
    progress_bar.close()
 
  return avg_loss
 

In [ ]:
# https://spell.ml/blog/mixed-precision-training-with-pytorch-Xuk7YBEAACAASJam
def train_validate(model, train_ml_df, valid_ml_df, epochs=EPOCHS):
  lowest_loss = int(1e09)
  lowest_after = 0
  for i in range(epochs):
    train_ml_df = train_ml_df.sample(frac=1)
    train_img_paths = [f"{DATA_IN_PATH}train/{x}.jpg" for x in train_ml_df["Id"].values]
    train_img_paths += [f"{DATA_IN_PATH}cropped_train/{x}.jpg" for x in train_ml_df["Id"].values]
    new_train_ml_df = pd.concat([train_ml_df, train_ml_df], axis=0)
    valid_ml_df = valid_ml_df.sample(frac=1)
    valid_img_paths = [f"{DATA_IN_PATH}train/{x}.jpg" for x in valid_ml_df["Id"].values]
    valid_img_paths += [f"{DATA_IN_PATH}cropped_train/{x}.jpg" for x in valid_ml_df["Id"].values]
    new_valid_ml_df = pd.concat([valid_ml_df, valid_ml_df], axis=0)
    train_dataset = PawpularDataset(
        image_paths=train_img_paths,
        dense_features=new_train_ml_df[DENSE_FEATURES].values,
        targets=new_train_ml_df.Pawpularity.values/100.0,
        augmentations=AUGMENTS
    )
    valid_dataset = PawpularDataset(
        image_paths=valid_img_paths,
        dense_features=new_valid_ml_df[DENSE_FEATURES].values,
        targets=new_valid_ml_df.Pawpularity.values/100.0,
        augmentations=AUGMENTS2
    )

    total_loss = 0.0
    batches = 0
    all_train_labels = np.array([]).astype(int)
    all_train_preds = np.array([]).astype(int)
    model.train()
    # https://betterprogramming.pub/how-to-make-your-pytorch-code-run-faster-93079f3c1f7b
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, 
                              num_workers = WORKERS, pin_memory = True)
    progress_bar = tqdm(train_loader, desc=f'[EPOCHS#{i}] Training: ', unit='epochs')
    total_cnt = 0
    correct_cnt = 0
    for mini_batch in progress_bar:
      optimizer.zero_grad() 
      inputs = {'image': mini_batch[0].to(device, dtype=torch.float), 
                'features': mini_batch[1].to(device, dtype=torch.float)}
      with autocast():
        outputs = model(**inputs)
        loss = loss_fn(outputs, mini_batch[2].to(device, dtype=torch.float))
      scaler.scale(loss).backward()
      # loss.backward()
      if DO_CLIP_GRAD:
        adaptive_clip_grad(model.parameters(), clip_factor=0.01, eps=1e-3, norm_type=2.0)

      scaler.step(optimizer)
      scaler.update()
      # optimizer.step()
      scheduler.step(loss)

      batches += 1
      total_loss += loss.item()
      avg_loss = total_loss / batches
      progress_bar.set_postfix_str('|train_loss|=%.4f' % (avg_loss))
      torch.cuda.empty_cache()

    del inputs, outputs
    gc.collect()
    torch.cuda.empty_cache()
    progress_bar.close()
 
    val_avg_loss = evaluate(model, valid_dataset, phrase='validation')
    # scheduler.step(val_avg_loss)
    # train loss(avg_loss)가 너무 작으면 오버 피팅 되므로 valid loss와 차이가 근소해야
    if val_avg_loss < lowest_loss: # and (abs(val_avg_loss - avg_loss) < 0.1 or val_avg_loss < avg_loss):
        # update if there is an improvement
        lowest_loss = val_avg_loss
        lowest_after = 0
        torch.save(model.state_dict(), SAVE_PRETRAINED_PATH)
    else:
        # model.load_state_dict(torch.load(SAVE_PRETRAINED_PATH))
        lowest_after += 1
        if lowest_after >= EARLY_STOP and EARLY_STOP > 0:
            break
 

## 훈련 수행

! 아직 오프라인으로는 timm이 모델 파일이 수행 안됨

https://www.kaggle.com/sreevishnudamodaran/siim-effnetv2-keras-study-train-tpu-cv0-805

In [ ]:
def implement_kfold_training(epochs: int = 20, 
                             lr: float = 1e-4, 
                             retricted_folds: list = [2,3,4], 
                             model_version: str = 'v1', lr_cycle: int = 1):
  kfold = KFold(n_splits=NUM_FOLDS, random_state=SEED, shuffle=True)
  for fold, (train_indices, val_indices) in enumerate(kfold.split(dataset_df)):
    if fold in retricted_folds:
      continue
    print(f"\nFold {fold + 1}/{NUM_FOLDS}")
    train_ml_df = dataset_df.iloc[train_indices]
    valid_ml_df = dataset_df.iloc[val_indices]
    model = PawpularModel3(model_name=MODEL_NAME, pretrained=True)
    global optimizer, scheduler, SAVE_PRETRAINED_PATH
    SAVE_PRETRAINED_PATH = f"/content/drive/MyDrive/Portfolio_for_AI/swin_f{fold}_{model_version}.bin"
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    # scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=0, verbose=False, min_lr=1e-6)
    # scheduler = get_cosine_schedule_with_warmup(
    #   optimizer,
    #   num_warmup_steps=0,
    #   num_training_steps=epochs * 284
    #   )
    scheduler = CosineAnnealingWarmRestarts(
        optimizer, T_0=5, eta_min=1e-7, last_epoch=-1)

    train_validate(model, train_ml_df, valid_ml_df, epochs)
    del model
    gc.collect()

if DO_KFOLD_TRAINING:
    implement_kfold_training(epochs=EPOCHS, lr=LEARNING_RATE, retricted_folds=[], model_version='petfinder_v1') 

## 6. 훈련 모델을 바탕으로 테스트

In [ ]:
if USE_SWIN_SVR:
    NUM_FOLDS = 10
    retricted_folds = []

    super_final_predictions = []
    super_final_predictions2 = []
    super_final_predictions3 = []
    super_final_predictions4 = []

    testset_df = pd.read_csv(f"{DATA_IN_PATH}test.csv")
    img_paths = [f"{DATA_IN_PATH}test/{x}.jpg" for x in testset_df["Id"].values]
    test_dataset = PawpularDataset2(
        image_paths=img_paths,
        dense_features=testset_df[DENSE_FEATURES].values,
        targets=np.ones(len(img_paths)),
        augmentations=AUGMENTS3,
    )
    test_dataset2 = PawpularDataset2(
        image_paths=img_paths,
        dense_features=testset_df[DENSE_FEATURES].values,
        targets=np.ones(len(img_paths)),
        augmentations=AUGMENTS4,
    )

    kfold = KFold(n_splits=NUM_FOLDS, random_state=SEED, shuffle=True)
    for fold, (train_indices, val_indices) in enumerate(kfold.split(dataset_df)):
        if fold in retricted_folds:
          continue
        name = f"SVR_fold_{fold}.pkl"
        print(f"\nFold {fold + 1}/{NUM_FOLDS}")
        train_ml_df = dataset_df.iloc[train_indices]
        valid_ml_df = dataset_df.iloc[val_indices]
        model = PawpularModel3(model_name=MODEL_NAME, pretrained=False)
        SAVE_PRETRAINED_PATH = f"{SAVE_MODEL_PATH}swin_f{fold}.bin"
        model.load(SAVE_PRETRAINED_PATH, device="cuda", weights_only=True)

        if DO_SVR_TRAIN:
            train_ml_df = train_ml_df.sample(frac=1)
            train_img_paths = [f"{DATA_IN_PATH}train/{x}.jpg" for x in train_ml_df["Id"].values]
            train_img_paths += [f"{DATA_IN_PATH}cropped_train/{x}.jpg" for x in train_ml_df["Id"].values]
            new_train_ml_df = pd.concat([train_ml_df, train_ml_df], axis=0)
            train_dataset = PawpularDataset(
                image_paths=train_img_paths,
                dense_features=new_train_ml_df[DENSE_FEATURES].values,
                targets=new_train_ml_df.Pawpularity.values/100.0,
                augmentations=AUGMENTS
            )

            train_predictions = model.predict(train_dataset, batch_size=2*BATCH_SIZE, n_jobs=-1)
            embed = np.array([]).reshape((0,128+12))
            for preds in train_predictions:
                embed = np.concatenate([embed, preds[:,1:]],axis=0)

            # fit RAPID SVR
            clf = SVR(C=20.0)
            clf.fit(embed.astype('float32'), df_train.Pawpularity.values.astype('int32'))
            pickle.dump(clf, open(SAVE_MODEL_PATH+name, "wb"))

        else:
            # LOAD RAPIDS SVR 
            print('Loading SVR...',SAVE_MODEL_PATH+name)
            clf = pickle.load(open(SAVE_MODEL_PATH+name, "rb"))

        print('Predicting test...')
        test_predictions = model.predict(test_dataset, batch_size=2*BATCH_SIZE, n_jobs=-1)
        test_predictions2 = model.predict(test_dataset2, batch_size=2*BATCH_SIZE, n_jobs=-1)

        # AUGMENTS3 version
        final_test_predictions = []
        embed = np.array([]).reshape((0,128+12))
        for preds in test_predictions: #tqdm
            final_test_predictions.extend(preds[:,:1].ravel().tolist())
            embed = np.concatenate([embed,preds[:,1:]],axis=0)

        final_test_predictions = [sigmoid(x) * 100 for x in final_test_predictions]
        final_test_predictions2 = clf.predict(embed)
        super_final_predictions.append(final_test_predictions)
        super_final_predictions2.append(final_test_predictions2)

        # AUGMENTS4 version
        final_test_predictions = []
        embed = np.array([]).reshape((0,128+12))
        for preds in test_predictions2: #tqdm
            final_test_predictions.extend(preds[:,:1].ravel().tolist())
            embed = np.concatenate([embed,preds[:,1:]],axis=0)

        final_test_predictions = [sigmoid(x) * 100 for x in final_test_predictions]
        final_test_predictions2 = clf.predict(embed)
        super_final_predictions3.append(final_test_predictions)
        super_final_predictions4.append(final_test_predictions2)


    # 최종 submission 파일 제출
    super_final_predictions = np.mean(np.column_stack(super_final_predictions), axis=1)
    super_final_predictions2 = np.mean(np.column_stack(super_final_predictions2), axis=1)
    super_final_predictions3 = np.mean(np.column_stack(super_final_predictions3), axis=1)
    super_final_predictions4 = np.mean(np.column_stack(super_final_predictions4), axis=1)
    # testset_df["Pawpularity"] = 0.8*super_final_predictions + 0.2*super_final_predictions2
    # testset_df["Pawpularity"] = 0.64*super_final_predictions + 0.16*super_final_predictions2 + 0.16*super_final_predictions3 + 0.04*super_final_predictions4
    preds1 = 0.72*super_final_predictions + 0.18*super_final_predictions2 + 0.08*super_final_predictions3 + 0.02*super_final_predictions4

## 7. FastAI를 활용한 모델 예측

In [ ]:
import sys
sys.path.append("../input/timmmaster")
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
from timm import create_model
from timm.data.mixup import Mixup
from fastai.vision.all import *

if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'
!cp '../input/timmswin/swin_large_patch4_window12_384_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window12_384_22kto1k.pth'
# !cp '../input/beit-large-patch16/beit_large_patch16_384_pt22k_ft22kto1k.pth' '/root/.cache/torch/hub/checkpoints/beit_large_patch16_384_pt22k_ft22kto1k.pth'
!cp '../input/beit-large-patch16/beit_large_patch16_224_pt22k_ft22kto1k.pth' '/root/.cache/torch/hub/checkpoints/beit_large_patch16_224_pt22k_ft22kto1k.pth'

USE_FASTAI_SWIN = True
BATCH_SIZE = 8 #32 Only if inference
N_FOLDS = 5
SEED=999
KFOLD_TRAIN = False
MODEL_NAME = 'swin_large_patch4_window7_224'
IMAGE_SIZE = 384 # swin large should be 224
PRETRAINED_MODEL_PATH = '../input/petfinder-pretrained-model'
PRETRAINED_MODEL_NAME = 'swin_large_patch4_window12_384_rmse15p30'
EPOCHS = 5 
WORKERS = 4
LEARNING_RATE = 2e-5
# TRAINED_MODEL_PATH = "../input/petfinder-trained-model-rmse17p41-v3"
# TRAINED_MODEL_PATH = "../input/petfinder-trained-model-rmse16p86-v4"
TRAINED_MODEL_PATH = "../input/petfinder-trained-model-16p57-384-v1"

set_seed(SEED, reproducible=True)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

dataset_path = Path('../input/petfinder-pawpularity-score/')
dataset_path.ls()

train_df = pd.read_csv(dataset_path/'train.csv')
train_df.head()

train_df['path'] = train_df['Id'].map(lambda x:str(dataset_path/'train'/x)+'.jpg')
train_df = train_df.drop(columns=['Id'])
train_df = train_df.sample(frac=1).reset_index(drop=True) #shuffle dataframe
train_df.head()

len_df = len(train_df)
print(f"There are {len_df} images")

train_df['norm_score'] = train_df['Pawpularity']/100

#Sturges' rule
num_bins = int(np.floor(1+(3.3)*(np.log2(len(train_df)))))

train_df['bins'] = pd.cut(train_df['norm_score'], bins=num_bins, labels=False)
train_df['bins'].hist()

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold


def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

def get_data(fold):
#     train_df_no_val = train_df.query(f'fold != {fold}')
#     train_df_val = train_df.query(f'fold == {fold}')
    
#     train_df_bal = pd.concat([train_df_no_val,train_df_val.sample(frac=1).reset_index(drop=True)])
    train_df_f = train_df.copy()
    # add is_valid for validation fold
    train_df_f['is_valid'] = (train_df_f['fold'] == fold)
    
    dls = ImageDataLoaders.from_df(train_df_f, #pass in train DataFrame
#                                valid_pct=0.2, #80-20 train-validation random split
                                   valid_col='is_valid', #
                                   seed=SEED, #seed
                                   fn_col='path', #filename/path is in the second column of the DataFrame
                                   label_col='norm_score', #label is in the first column of the DataFrame
                                   y_block=RegressionBlock, #The type of target
                                   bs=BATCH_SIZE, #pass in batch size
                                   num_workers=WORKERS,
                                   item_tfms=Resize(IMAGE_SIZE), #pass in item_tfms
                                   batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()])) #pass in batch_tfms
    
    return dls

def get_learner(fold_num):
    data = get_data(fold_num)
    
    model = create_model(MODEL_NAME, pretrained=True, num_classes=data.c)

    # learn = Learner(data, model, loss_func=BCEWithLogitsLossFlat(), metrics=petfinder_rmse).to_fp16()
    learn = Learner(data, model, loss_func=BCEWithLogitsLossFlat(), metrics=petfinder_rmse, cbs=[MixUp(0.2)]).to_fp16()
    
    return learn


train_df['fold'] = -1

strat_kfold = StratifiedKFold(n_splits=N_FOLDS, random_state=SEED, shuffle=True)
for i, (_, train_index) in enumerate(strat_kfold.split(train_df.index, train_df['bins'])):
    train_df.iloc[train_index, -1] = i
    
train_df['fold'] = train_df['fold'].astype('int')


if USE_FASTAI_SWIN:
    BATCH_SIZE = 8
    #Valid Kfolder size
    the_data = get_data(0)
    assert (len(the_data.train) + len(the_data.valid)) == (len(train_df)//BATCH_SIZE)

    test_df = pd.read_csv(dataset_path/'test.csv')
    test_df.head()

    test_df['Pawpularity'] = [1]*len(test_df)
    test_df['path'] = test_df['Id'].map(lambda x:str(dataset_path/'test'/x)+'.jpg')
    test_df = test_df.drop(columns=['Id'])
    train_df['norm_score'] = train_df['Pawpularity']/100
    
#     LEARNING_RATE_SUGGESTED = []
#     for i in range(5):
#         learner = get_learner(fold_num=i)
#         lr_sug = learner.lr_find(end_lr=3e-2)
#         LEARNING_RATE_SUGGESTED.append(lr_sug.valley)
#         print(f'Fold{i} suggested lr: ', lr_sug.valley)

In [ ]:
# self training을 위한 준비: 추가 학습 데이터 만들기
# img_path_list = []
# test2_dir_path = '../input/petfinder-adoption-prediction/test_images'
# for image_file in os.listdir(test2_dir_path):
#     img_path_list.append(test2_dir_path + '/' + image_file)
    
# train2_dir_path = '../input/petfinder-adoption-prediction/train_images'
# for image_file in os.listdir(train2_dir_path):
#     img_path_list.append(train2_dir_path + '/' + image_file)

# # img_path_list = img_path_list[:10]
# test_df = pd.DataFrame({'path': img_path_list, 'Pawpularity': [1]*len(img_path_list)})
# test_df
# self training을 위한 준비: 추가 학습셋으로 훈련하기
# pretrain_df = pd.read_csv('~submission.csv')
# pretrain_df['fold'] = 1
# pretrain_df = pretrain_df.rename(columns={'Pawpularity': 'norm_score'})
# pretrain_df = pretrain_df[['path', 'norm_score', 'fold']]
# train_df['fold'] = 0
# train_df = train_df[['path', 'norm_score', 'fold']]
# test_df = train_df[['path', 'norm_score']]
# train_df = pd.concat([pretrain_df, train_df], axis=0)
# N_FOLDS = 1

In [ ]:
import gc
# LEARNING_RATE_SUGGESTED = [5.477225568029098e-05, 2.9154431103961542e-05, 0.00024877983378246427, 0.00010290031059412286, 0.00021930268849246204]

def fastai_swin_kfolds():
    all_preds = []
    global TRAINED_MODEL_PATH, TRAINED_MODEL_PATH2
    for i in range(N_FOLDS):
        if i in [3, 4] and 'beit' in TRAINED_MODEL_PATH:
            TRAINED_MODEL_PATH = TRAINED_MODEL_PATH2
        print(f'Fold {i} results')
        learn = get_learner(fold_num=i)
        if KFOLD_TRAIN:
            original_model_dir = learn.model_dir
            learn.model_dir = PRETRAINED_MODEL_PATH
            learn.load(PRETRAINED_MODEL_NAME)
            learn.model_dir = original_model_dir
            learn.fit_one_cycle(EPOCHS, LEARNING_RATE, cbs=[SaveModelCallback(), EarlyStoppingCallback(monitor='petfinder_rmse', comp=np.less, patience=2)]) 

            learn.recorder.plot_loss()

            learn = learn.to_fp32()
            learn.export(f'model_fold_{i}.pkl')
            learn.save(f'model_fold_{i}.pkl')

        else:
            learn.model_dir = TRAINED_MODEL_PATH
            learn.load(f'model_fold_{i}.pkl')

        dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
                                       valid_pct=0.2, #80-20 train-validation random split
                                       seed=SEED, #seed
                                       fn_col='path', #filename/path is in the second column of the DataFrame
                                       label_col='norm_score', #label is in the first column of the DataFrame
                                       y_block=RegressionBlock, #The type of target
                                       bs=BATCH_SIZE, #pass in batch size
                                       num_workers=WORKERS,
                                       item_tfms=Resize(IMAGE_SIZE), #pass in item_tfms
                                       batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()])) 



        test_dl = dls.test_dl(test_df)

        preds, _ = learn.tta(dl=test_dl, n=5, beta=0)

        all_preds.append(preds)

        del learn

        torch.cuda.empty_cache()

        gc.collect()

    preds = np.mean(np.stack(all_preds), axis=0)
    # preds = preds*100
    return preds

if USE_FASTAI_SWIN:
    KFOLD_TRAIN = True
    BATCH_SIZE = 8 # when inference. train 시는 batch size 8로 해야
    MODEL_NAME = 'swin_large_patch4_window12_384'
    IMAGE_SIZE = 384 
    TRAINED_MODEL_PATH = "../input/petfinder-trained-model-16p52-384-v2"
    preds2 = fastai_swin_kfolds()
    
#     MODEL_NAME = 'swin_large_patch4_window7_224'
#     IMAGE_SIZE = 224 
#     TRAINED_MODEL_PATH = "../input/petfinder-trained-model-16p67-224-v2"
#     preds3 = fastai_swin_kfolds()
    
#     MODEL_NAME = 'beit_large_patch16_224'
#     IMAGE_SIZE = 224 
#     TRAINED_MODEL_PATH = "../input/petfinder-trained-model-beit-224-rmse16p83-v1"
#     TRAINED_MODEL_PATH2 = "../input/petfinder-trained-model-beit-224-rmse16p85-v1-2"
#     preds4 = fastai_swin_kfolds()

In [ ]:
# sample_df = pd.read_csv(dataset_path/'sample_submission.csv')
# # sample_df['Pawpularity'] = preds1*0.10 + preds2*0.55 + preds3*0.15 + preds4*0.20
# # LB version
# # sample_df['Pawpularity'] = preds1*0.02 + preds2*0.04 + preds3*0.85 + preds4*0.09
# # Val version
# sample_df['Pawpularity'] = preds1*0.05 + preds2*0.55 + preds3*0.16 + preds4*0.24
# # sample_df['Pawpularity'] = preds2
# sample_df.to_csv('submission.csv',index=False)
# print(sample_df.head())

In [ ]:
# test_df['Pawpularity'] = preds2*0.65 + preds3*0.20 + preds4*0.15
test_df['Pawpularity'] = preds2
test_df.to_csv('submission.csv',index=False)
print(test_df.head())